In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score


In [2]:
df_org = pd.read_csv("train.csv",encoding="latin1")
df = df_org[["selected_text","sentiment"]]
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df["sentiment"] = encoder.fit_transform(df["sentiment"])

C:\Users\Aspire_Lays\AppData\Local\Temp\ipykernel_13328\3583909566.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sentiment"] = encoder.fit_transform(df["sentiment"])


In [6]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(df["selected_text"], df["sentiment"], test_size=0.2, random_state=42)

# Tokenization
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")  # Use top 5000 words, mark unknown words as <OOV>
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding sequences to equal length
max_length = max(len(seq) for seq in X_train_seq)
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding="post")
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding="post")

In [7]:
# Define RNN model
model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=max_length),  # Word embeddings
    SimpleRNN(64, return_sequences=False),  # RNN layer
    Dense(32, activation="relu"),  # Fully connected layer
    Dense(1, activation="sigmoid")  # Output layer (binary classification)
])

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train_padded, np.array(y_train), epochs=10, batch_size=2, validation_data=(X_test_padded, np.array(y_test)))


Epoch 1/10
10992/10992 [==============================] - 88s 8ms/step - loss: -7.0674 - accuracy: 0.4043 - val_loss: -17.4234 - val_accuracy: 0.4057
Epoch 2/10
10992/10992 [==============================] - 87s 8ms/step - loss: -38.2716 - accuracy: 0.4043 - val_loss: -55.0097 - val_accuracy: 0.4057
Epoch 3/10
10992/10992 [==============================] - 88s 8ms/step - loss: -82.9863 - accuracy: 0.4043 - val_loss: -101.7079 - val_accuracy: 0.4057
Epoch 4/10
10992/10992 [==============================] - 91s 8ms/step - loss: -146.6309 - accuracy: 0.4043 - val_loss: -166.9619 - val_accuracy: 0.4057
Epoch 5/10
10992/10992 [==============================] - 89s 8ms/step - loss: -228.8760 - accuracy: 0.4043 - val_loss: -249.7553 - val_accuracy: 0.4057
Epoch 6/10
10992/10992 [==============================] - 92s 8ms/step - loss: -330.1341 - accuracy: 0.4043 - val_loss: -349.0474 - val_accuracy: 0.4057
Epoch 7/10
10992/10992 [==============================] - 91s 8ms/step - loss: -449.4839

In [8]:
# Predict on test data
y_pred = (model.predict(X_test_padded) > 0.5).astype("int32")

# Print classification report
print(classification_report(y_test, y_pred))


172/172 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1562
           1       0.41      1.00      0.58      2230
           2       0.00      0.00      0.00      1705

    accuracy                           0.41      5497
   macro avg       0.14      0.33      0.19      5497
weighted avg       0.16      0.41      0.23      5497



c:\Users\Aspire_Lays\anaconda3\envs\ai\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aspire_Lays\anaconda3\envs\ai\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aspire_Lays\anaconda3\envs\ai\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
print(accuracy_score(y_test,y_pred))

0.405675823176278
